### 講習5 --- astropy.io.fitsの基本  

FITSデータの読み書きが可能なモジュールです。 元々、pyfitsと呼ばれていたものです。astropyに吸収されてそちらで統一的に管理されるようです。

PyRAFでもFITSデータの読み書きはできるのですが、ndarrayとしてデータを読み出しておくとより自由な処理を行うことができます。例えば、IRAFには存在しない統計アルゴリズムを適用する、あるいは、matplotlibを用いた自由度の高い可視化などです。  

まずastropy.ioからfitsをimportしておきます。

In [1]:
from astropy.io import fits

#### 基本のgetdata
まずは簡単なデータ読み出し方法。 **fits.getdata( )**です。 ピクセル値をndarrayとして読み出します。

In [2]:
data = fits.getdata('idfs0011.fits')

In [3]:
data

array([[ 28.25488281, -10.32543945,   0.        , ...,   9.51599121,
         16.44238281,   0.        ],
       [-13.28271484,  -6.83496094,  35.28259277, ...,  -1.86853027,
          0.        ,  10.52661133],
       [  0.        , -12.53417969,   3.80444336, ...,   3.79370117,
          1.87597656,  -7.6472168 ],
       ..., 
       [-28.76989746,  -6.64709473,  45.30151367, ...,  -3.17895508,
        -27.85888672,   7.49157715],
       [  4.18920898,  12.38256836, -18.12219238, ...,  -5.96862793,
        -11.26855469,   6.64306641],
       [  0.        ,  10.22607422, -29.54064941, ...,  13.56323242,
         -9.09277344,   2.10144043]], dtype=float32)

特定のピクセルの値を書き出してみます。

In [4]:
print (data[0, 5])

27.6495


これは(x, y) = (6, 1)のピクセルであることに注意してください。
- Pythonではインデックスは0から始まる
- numpyのndarrayではFITSの二次元データは(y, x)の順

idf0011.fitsをds9で直接オープンして、(6, 1)のピクセル値を読み取って確認してみてください。(ds9のメニューバーのanalysis>pixel table を使うと便利です。)

#### numpyの関数で処理

FITSの二次元データはndarrayとして読み出されるので、numpyの関数が使えます。

In [5]:
import numpy as np

In [6]:
np.median(data), np.std(data), np.min(data), np.max(data)

(0.0, 199.40973, -82567.516, 47761.172)

#### _補足_
上限値と下限値を適切に設定し、3-sigmaクリッップすると標準偏差がもう少し小さくなります。


In [7]:
xx = np.where(np.fabs(data) < 1000)  #  値が+/-1000以内のところだけ抽出
med = np.median(data[xx])
std = np.std(data[xx])

xx = np.where(np.fabs(data - med)  < 3 * std)  #  3-sigma clip
med = np.median(data[xx])
std = np.std(data[xx])
print ('{:.2f} {:.2f}'.format(med, std))   #   format()関数使ってみた


0.00 20.35


#### データの加工  

IRAFは長期間にわたって多くの人たちに利用されてきたので、プログラムのバグがかなり修正されてきています。
こういうのを、いい意味で「枯れたシステム」ということがあります。
枯れたシステムのIRAFの関数はとてもありがたい存在なのですが、画像の足し算、引き算、割り算くらいなら、
IRAFを介さずにpythonのモジュールだけで済ませるほうが便利なことが多いです。  

astropy.io.fitsでFITSをndarrayとして読み取り、講習2で行った1次処理のダーク引き、フラット割り、スカイバイアス引きの部分を、
ndarrayの演算として行ってみます。 

In [8]:
rdata = fits.getdata('sample_data2/ir0011.fits')
dark = fits.getdata('dark5.fits')
flat = fits.getdata('flat.fits')
skybias = fits.getdata('skybias1.fits')

0での割り算に注意します。IRAFでは何か裏で工夫してたのですが、ここでは明示して処理しておきます。  
ピクセル値が0のところを-9999におきかえます。

In [9]:
xx = np.where( np.abs(flat) == 0.0 ) 
flat[xx] = -9999.

IRAFの場合と違い、一行で処理の式が書けます。また、途中の暫定的なファイルを介在せずに処理ができます。

In [10]:
ndata= ( rdata - dark ) / flat - skybias

結果をファイルに書き出すときには、**fits.writeto()**関数を使います。  
**fits.writeto('ファイル名.fits', ndarrayデータ)** 

In [11]:
fits.writeto('new0011.fits', ndata)

はたして、idfs0011.fitsと同じでしょうか？  ds9で見比べてみてください。

あるいは、ちゃんと比較するには、(idfs0011.fitsのデータは上でdataとして読み込んでいるので）、dataとndataの差分の統計をみてみましょう。

In [12]:
diff = data - ndata

In [13]:
np.median(diff), np.std(diff)

(0.0, 2.7135166e-06)

IRAFで処理したものも、非IRAFで処理したものも同じ結果が得られたようですね。